In [1]:
##This will have all 3 parts to the questions and will be posted for all 3 attachments

In [2]:
##Question 1 Cleaning the Data

In [3]:
! pip install beautifulsoup4

In [5]:
import urllib.request

In [6]:
url='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'

In [7]:
html_doc=urllib.request.urlopen(url)

In [8]:
from bs4 import BeautifulSoup

In [9]:
b_soup=BeautifulSoup(html_doc)

In [28]:
print(b_soup.prettify())

<!DOCTYPE html>
<html class="client-nojs" dir="ltr" lang="en">
 <head>
  <meta charset="utf-8"/>
  <title>
   List of postal codes of Canada: M - Wikipedia
  </title>
  <script>
   document.documentElement.className="client-js";RLCONF={"wgBreakFrames":!1,"wgSeparatorTransformTable":["",""],"wgDigitTransformTable":["",""],"wgDefaultDateFormat":"dmy","wgMonthNames":["","January","February","March","April","May","June","July","August","September","October","November","December"],"wgRequestId":"b9ab4b8a-cc06-4246-8f43-17bd160e26a6","wgCSPNonce":!1,"wgCanonicalNamespace":"","wgCanonicalSpecialPageName":!1,"wgNamespaceNumber":0,"wgPageName":"List_of_postal_codes_of_Canada:_M","wgTitle":"List of postal codes of Canada: M","wgCurRevisionId":979555370,"wgRevisionId":979555370,"wgArticleId":539066,"wgIsArticle":!0,"wgIsRedirect":!1,"wgAction":"view","wgUserName":null,"wgUserGroups":["*"],"wgCategories":["Articles with short description","Short description is different from Wikidata","Communicati

In [29]:
data=b_soup.find('table',class_='wikitable sortable')
data

import pandas as pd
import numpy as np

In [30]:
columns=[]
for column in data.find_all('th'):
    columns.append(column.get_text())

In [31]:
columns

['Postal Code\n', 'Borough\n', 'Neighbourhood\n']

In [32]:
columns[0]='PostalCode'
columns[2]='Neighborhood'

In [33]:
columns

['PostalCode', 'Borough\n', 'Neighborhood']

In [34]:
df=pd.DataFrame(columns=columns)
df

,PostalCode,Borough\n,Neighborhood


In [35]:
A=[]
B=[]
C=[]
for row in data.find_all("tr"):
    elements=row.find_all('td')
    if len(elements)==3:
        A.append(elements[0].get_text().strip())
        B.append(elements[1].get_text().strip())
        C.append(elements[2].get_text().strip())

In [36]:
df['PostalCode']=A
df['Borough']=B
df['Neighborhood']=C

In [37]:
df.head()

,PostalCode,Borough\n,Neighborhood,Borough
0,M1A,NaN,Not assigned,Not assigned
1,M2A,NaN,Not assigned,Not assigned
2,M3A,NaN,Parkwoods,North York
3,M4A,NaN,Victoria Village,North York
4,M5A,NaN,"Regent Park, Harbourfront",Downtown Toronto


In [38]:
df.shape

(180, 4)

In [39]:
df.to_csv('TorontoDataset.csv')

In [40]:
## Question 2 Geographical Coordinates

In [41]:
import pandas as pd
import numpy as np

In [42]:
df=pd.read_csv('TorontoDataset.csv')

In [43]:
df.head()

,Unnamed: 0,PostalCode,Borough\n,Neighborhood,Borough
0,0,M1A,NaN,Not assigned,Not assigned
1,1,M2A,NaN,Not assigned,Not assigned
2,2,M3A,NaN,Parkwoods,North York
3,3,M4A,NaN,Victoria Village,North York
4,4,M5A,NaN,"Regent Park, Harbourfront",Downtown Toronto


In [44]:
df.drop('Unnamed: 0',axis=1,inplace=True)

In [45]:
df.head()

,PostalCode,Borough\n,Neighborhood,Borough
0,M1A,NaN,Not assigned,Not assigned
1,M2A,NaN,Not assigned,Not assigned
2,M3A,NaN,Parkwoods,North York
3,M4A,NaN,Victoria Village,North York
4,M5A,NaN,"Regent Park, Harbourfront",Downtown Toronto


In [46]:
df_geo=pd.read_csv('https://cocl.us/Geospatial_data')

In [47]:

df_geo.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [48]:
df_geo.shape

(103, 3)

In [49]:
df_geo=df_geo.rename({'Postal Code':'PostalCode'},axis=1)

In [50]:
df_geo.head()

,PostalCode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [51]:
df=df.join(df_geo.set_index('PostalCode'),on='PostalCode')


In [52]:
df.head()

,PostalCode,Borough\n,Neighborhood,Borough,Latitude,Longitude
0,M1A,NaN,Not assigned,Not assigned,NaN,NaN
1,M2A,NaN,Not assigned,Not assigned,NaN,NaN
2,M3A,NaN,Parkwoods,North York,43.753259,-79.329656
3,M4A,NaN,Victoria Village,North York,43.725882,-79.315572
4,M5A,NaN,"Regent Park, Harbourfront",Downtown Toronto,43.654260,-79.360636


In [53]:
df.to_csv("TorontoGeoData.csv")

In [54]:
## Question 3 Exploring and Clustering the Neighborhood in Toronto

In [55]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans
!pip install folium
import folium # map rendering library

print('Libraries imported.')

     |████████████████████████████████| 93 kB 2.4 MB/s eta 0:00:011
Libraries imported.


In [56]:

df=pd.read_csv('TorontoGeoData.csv')

In [57]:
df.head()

,Unnamed: 0,PostalCode,Borough\n,Neighborhood,Borough,Latitude,Longitude
0,0,M1A,NaN,Not assigned,Not assigned,NaN,NaN
1,1,M2A,NaN,Not assigned,Not assigned,NaN,NaN
2,2,M3A,NaN,Parkwoods,North York,43.753259,-79.329656
3,3,M4A,NaN,Victoria Village,North York,43.725882,-79.315572
4,4,M5A,NaN,"Regent Park, Harbourfront",Downtown Toronto,43.654260,-79.360636


In [58]:
toronto_add='Toronto, Ontario' #CITY ADDRESS
geolocator=Nominatim(user_agent='toronto_explorer')
location=geolocator.geocode(toronto_add)
latitude=location.latitude
longitude=location.longitude
print("The coordinates of Toronto are {},{}".format(latitude,longitude))

The coordinates of Toronto are 43.6534817,-79.3839347


In [59]:
Toronto_map=folium.Map([latitude,longitude],zoom_start=10)
for lat,long,postalcode,borough,neighborhood in zip(df['Latitude'],df['Longitude'],df['PostalCode'],df['Borough'],df['Neighborhood']):
    label='Postal Code:{},Neighborhoods:{},Borough:{}'.format(postalcode,neighborhood,borough)
    label=folium.Popup(label,parse_html=True)
    folium.CircleMarker([lat,long],radius=5,color='blue',popup=label,
                        fill=True,fill_color='#3186cc',fill_opacity=0.7,
                        parse_html=True).add_to(Toronto_map)

ValueError: Location values cannot contain NaNs.

In [60]:
Toronto_map

In [61]:
import requests# library to handle requests
import json # library to handle JSON files
from pandas.io.json import json_normalize# tranform JSON file into a pandas dataframe

In [66]:
limit=100
def getNearbyVenues(codes,names,latitudes,longitudes,radius=500):
    venues_list=[]
    for code,name,lat,lng in zip(codes,names,latitudes,longitudes):
        url='https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID,CLIENT_SECRET,VERSION,lat,lng,radius,limit)
        results=requests.get(url).json()['response']['groups'][0]['items']
        venues_list.append([(code,name,lat,lng,v['venue']['name'],v['venue']['location']['lat'],
                          v['venue']['location']['lng'],
                          v['venue']['categories'][0]['name'])for v in results])
    nearby_venues=pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns=['PostalCode','Neighborhood','Neighborhood Latitude',
                          'Neighborhood Longitude','Venue name','Venue Latitude','Venue Longitude','Venue Category']
    return(nearby_venues)

In [67]:
Toronto_Venues=getNearbyVenues(codes=df['PostalCode'],
                               names=df['Neighborhood'],
                               latitudes=df['Latitude'],
                               longitudes=df['Longitude'],radius=500)

NameError: name 'CLIENT_ID' is not defined

In [68]:
Toronto_Venues.head()

NameError: name 'Toronto_Venues' is not defined

In [65]:
Toronto_Venues.shape

NameError: name 'Toronto_Venues' is not defined

In [69]:
Toronto_Venues.groupby('PostalCode').count()[['Venue name']]

NameError: name 'Toronto_Venues' is not defined

In [70]:

print('There are {} unique venue categories'.format(len(Toronto_Venues['Venue Category'].unique())))

NameError: name 'Toronto_Venues' is not defined

In [71]:
Toronto_onehot=pd.get_dummies(Toronto_Venues['Venue Category'],prefix='',prefix_sep='')#one hot encoding
#Add Neighborhood and PostalCode column back to Toronto_onehot dataframe
Toronto_onehot['PostalCode']=Toronto_Venues['PostalCode']
columns=[Toronto_onehot.columns[-1]]+list(Toronto_onehot.columns[:-1])
Toronto_onehot=Toronto_onehot[columns]
Toronto_onehot.head()

NameError: name 'Toronto_Venues' is not defined

In [72]:
Toronto_onehot.shape

NameError: name 'Toronto_onehot' is not defined

In [73]:
X=Toronto_Venues['Neighborhood']

NameError: name 'Toronto_Venues' is not defined

In [74]:
Toronto_onehot[-1]=X

NameError: name 'X' is not defined

In [75]:
Toronto_onehot=Toronto_onehot.rename({-1:'Neighborhoods'},axis=1)

NameError: name 'Toronto_onehot' is not defined

In [76]:
Toronto_onehot.head()

NameError: name 'Toronto_onehot' is not defined

In [77]:
num=5
for code,nbd in zip(Toronto_grouped['PostalCode'][0:10],Toronto_grouped['Neighborhoods'][0:10]):
    print('----'+code+':'+nbd+'----')
    temp=Toronto_grouped[Toronto_grouped['PostalCode']==code].T.reset_index()
    temp=temp.iloc[2:]
    temp.columns=['venue category','freq']
    temp['freq']=temp['freq'].astype(float)
    temp.sort_values(by=['freq'],ascending=False,inplace=True)
    temp=temp.round({'freq':2})
    print(temp.reset_index(drop=True).head(num))

NameError: name 'Toronto_grouped' is not defined

In [78]:

def most_common_venues(row,num_of_values):
    row_sorted=row.iloc[2:].sort_values(ascending=False)
    return row_sorted.index.values[0:num_of_values]

In [79]:
num_of_values=10
indicators=['st','nd','rd']
columns=['PostalCode','Neighborhoods']
for ind in np.arange(num_of_values):
    try:
        columns.append('{}{} Most common venue'.format(ind+1,indicators[ind]))
    except:
        columns.append('{}th Most common venue'.format(ind+1))

In [80]:
Toronto_top=pd.DataFrame(columns=columns)

In [81]:
Toronto_top['PostalCode']=Toronto_grouped['PostalCode']
Toronto_top['Neighborhoods']=Toronto_grouped['Neighborhoods']

NameError: name 'Toronto_grouped' is not defined

In [82]:
for ind in np.arange(Toronto_grouped.shape[0]):
    Toronto_top.iloc[ind][2:]=most_common_venues(Toronto_grouped.iloc[ind],num_of_values)

NameError: name 'Toronto_grouped' is not defined

In [83]:
Toronto_top.head()

,PostalCode,Neighborhoods,1st Most common venue,2nd Most common venue,3rd Most common venue,4th Most common venue,5th Most common venue,6th Most common venue,7th Most common venue,8th Most common venue,9th Most common venue,10th Most common venue


In [84]:
Toronto_grouped.drop(['PostalCode','Neighborhoods'],axis=1,inplace=True)
Toronto_grouped.head()

NameError: name 'Toronto_grouped' is not defined

In [85]:

k=5 # number of clusters
k_means=KMeans(n_clusters=k,random_state=0)
k_means.fit(Toronto_grouped)


NameError: name 'Toronto_grouped' is not defined

In [86]:

k_means.labels_[0:10]

AttributeError: 'KMeans' object has no attribute 'labels_'

In [87]:
Toronto_top.insert(0,' Cluster labels',k_means.labels_)

AttributeError: 'KMeans' object has no attribute 'labels_'

In [88]:
Toronto_top.head()

,PostalCode,Neighborhoods,1st Most common venue,2nd Most common venue,3rd Most common venue,4th Most common venue,5th Most common venue,6th Most common venue,7th Most common venue,8th Most common venue,9th Most common venue,10th Most common venue


In [89]:
Toronto_data=df
Toronto_top=Toronto_top.join(Toronto_data.set_index('PostalCode'),on=['PostalCode'])

In [90]:
Toronto_top.head()

,PostalCode,Neighborhoods,1st Most common venue,2nd Most common venue,3rd Most common venue,4th Most common venue,5th Most common venue,6th Most common venue,7th Most common venue,8th Most common venue,9th Most common venue,10th Most common venue,Unnamed: 0,Borough\n,Neighborhood,Borough,Latitude,Longitude


In [91]:
Toronto_top.drop(['Neighborhood'],axis=1,inplace=True)

In [92]:
Toronto_top.head()

,PostalCode,Neighborhoods,1st Most common venue,2nd Most common venue,3rd Most common venue,4th Most common venue,5th Most common venue,6th Most common venue,7th Most common venue,8th Most common venue,9th Most common venue,10th Most common venue,Unnamed: 0,Borough\n,Borough,Latitude,Longitude


In [93]:
map_clusters=folium.Map([latitude,longitude],zoom_start=10)

In [94]:
x=np.arange(k)
ys = [i + x + (i*x)**2 for i in range(k)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

In [95]:
for code,nbd,lat,lng,clus in zip(Toronto_top['PostalCode'],Toronto_top['Neighborhoods'],Toronto_top['Latitude'],Toronto_top['Longitude'],Toronto_top[' Cluster labels']):
    label=folium.Popup('Postal Code:{};Neighborhoods:{};Cluster:{}'.format(code,nbd,clus),parse_html=True)
    folium.CircleMarker([lat,lng],popup=label,color=rainbow[clus-1],
                       fill=True,
                       fill_color=rainbow[clus-1],
                       fill_opacity=0.6,
                       radius=5).add_to(map_clusters)

KeyError: ' Cluster labels'

In [96]:
map_clusters

In [97]:
Toronto_top.loc[Toronto_top[' Cluster labels']==0,Toronto_top.columns[:-3]].reset_index(drop=True)

KeyError: ' Cluster labels'

In [98]:
Toronto_top.loc[Toronto_top[' Cluster labels']==1,Toronto_top.columns[:-3]].reset_index(drop=True)

KeyError: ' Cluster labels'

In [99]:
Toronto_top.loc[Toronto_top[' Cluster labels']==2,Toronto_top.columns[:-3]].reset_index(drop=True)

KeyError: ' Cluster labels'

In [100]:
Toronto_top.loc[Toronto_top[' Cluster labels']==3,Toronto_top.columns[:-3]].reset_index(drop=True)

KeyError: ' Cluster labels'

In [101]:
Toronto_top.loc[Toronto_top[' Cluster labels']==4,Toronto_top.columns[:-3]].reset_index(drop=True)

KeyError: ' Cluster labels'